<a href="https://colab.research.google.com/github/claudevandort/UAI-DeepLearning/blob/master/02_LogisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regresión Logística

In [1]:
# Librerías

import torch.nn as nn
import torch
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [8]:
# Cargar datos
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

n_samples, n_features = X.shape

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

sc = StandardScaler()
# Escalar datos de entrenamiento y entrenar
X_train = sc.fit_transform(X_train)
# Escalar datos de prueba
X_test = sc.transform(X_test)

# Convertir a tensores
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

# Convertir a vector columna
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

In [10]:
# Modelo
class Model(nn.Module):
  # Declarar las partes del modelo
  def __init__(self, n_input_features):
    super(Model, self).__init__()
    self.linear = nn.Linear(n_input_features, 1)
  
  # Calcular estimación
  def forward(self, x):
    y_pred = torch.sigmoid(self.linear(x))
    return y_pred

model = Model(n_features)

# Loss y optimizador
num_epochs = 100
learning_rate = 0.02
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [13]:
# Entrenamiento
for epoch in range(num_epochs):
  # Forward
  y_pred = model(X_train)
  loss = criterion(y_pred, y_train)

  # Actualizar gradientes
  loss.backward()
  optimizer.step()
  optimizer.zero_grad()

  if (epoch+1) % 10 == 0:
    print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')

# Comparar con el conjunto de pruebas
with torch.no_grad():
  y_predicted = model(X_test).round()
  acc = y_predicted.eq(y_test).sum() / float(y_test.shape[0])
  print(f'Tasa de acierto: {acc.item():.4f}')

epoch: 10, loss = 0.1431
epoch: 20, loss = 0.1407
epoch: 30, loss = 0.1384
epoch: 40, loss = 0.1363
epoch: 50, loss = 0.1344
epoch: 60, loss = 0.1325
epoch: 70, loss = 0.1308
epoch: 80, loss = 0.1291
epoch: 90, loss = 0.1276
epoch: 100, loss = 0.1261
Tasa de acierto: 0.9825


# Red neuronal poco profunda

In [1]:
# Librerías
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# Modelo neuronal
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(1, 2, bias=True)
    self.fc2 = nn.Linear(2, 1, bias=True)

  def forward(self, x):
    x = self.fc2(F.relu(self.fc1(x)))
    return x

net = Net().cuda()
print(net)
print(list(net.parameters()))

Net(
  (fc1): Linear(in_features=1, out_features=2, bias=True)
  (fc2): Linear(in_features=2, out_features=1, bias=True)
)
[Parameter containing:
tensor([[0.8578],
        [0.5111]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([ 0.7574, -0.4648], device='cuda:0', requires_grad=True), Parameter containing:
tensor([[0.0288, 0.5403]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([-0.2859], device='cuda:0', requires_grad=True)]


In [3]:
# Test run
input = Variable(torch.randn(1, 1, 1)).cuda()
print(input)

net.zero_grad()

out = net(input)
print(out)

tensor([[[0.5727]]], device='cuda:0')
tensor([[[-0.2499]]], device='cuda:0', grad_fn=<AddBackward0>)


In [4]:
# Datos de prueba
data = [(1,3), (2, 6), (3,9), (4, 12), (5, 15), (6,18)]

# Entrenar
import torch.optim as optim
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.4)
criterion = nn.MSELoss()

for epoch in range(300):
  for i, data2 in enumerate(data):
    X, Y = iter(data2)
    X, Y = Variable(torch.FloatTensor([X]), requires_grad=True).cuda(), Variable(torch.FloatTensor([Y]), requires_grad=False).cuda()
    optimizer.zero_grad()
    y_pred =  net(X)
    output = criterion(y_pred, Y)
    output.backward()
    optimizer.step()

  if (epoch % 20 == 0.0):
    print('Epoch {} - loss: {}'.format(epoch, output))

Epoch 0 - loss: 161.71078491210938
Epoch 20 - loss: 0.031853802502155304
Epoch 40 - loss: 0.016081761568784714
Epoch 60 - loss: 0.00808698683977127
Epoch 80 - loss: 0.004053282085806131
Epoch 100 - loss: 0.0020260370802134275
Epoch 120 - loss: 0.001010953914374113
Epoch 140 - loss: 0.0005038084927946329
Epoch 160 - loss: 0.00025074114091694355
Epoch 180 - loss: 0.00012467104534152895
Epoch 200 - loss: 6.196252070367336e-05
Epoch 220 - loss: 3.0764436814934015e-05
Epoch 240 - loss: 1.5303525287890807e-05
Epoch 260 - loss: 7.596190698677674e-06
Epoch 280 - loss: 3.7701247492805123e-06


In [5]:
print(list(net.parameters()), end='\n\n')

# Listar los atributos de los parámetros
for param in net.parameters():
    print(type(param.data), param.size())

[Parameter containing:
tensor([[1.3129],
        [1.2568]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([ 0.6998, -0.4591], device='cuda:0', requires_grad=True), Parameter containing:
tensor([[1.0834, 1.2546]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([-0.1788], device='cuda:0', requires_grad=True)]

<class 'torch.Tensor'> torch.Size([2, 1])
<class 'torch.Tensor'> torch.Size([2])
<class 'torch.Tensor'> torch.Size([1, 2])
<class 'torch.Tensor'> torch.Size([1])
